In [33]:
import numpy as np
import matplotlib.pyplot as plt
from itertools import compress

import mne
from mne.datasets import sample

# the following import is required for matplotlib < 3.2:
from mpl_toolkits.mplot3d import Axes3D  # noqa
from pathlib import Path
import mne
import mne_nirs

import scipy.io as sio
import pandas as pd
import h5py as h 

import snirf as s 


raw = r"C:\Users\dalto\Downloads\project\sourcedata_lm\sub-10\ses-01\nirs\sub-10_ses-01_task-wings_nirs.snirf.snirf"

#imports fsaverage model brain components
subjects_dir = mne.datasets.sample.data_path() / 'subjects'
mne.datasets.fetch_hcp_mmp_parcellation(subjects_dir=subjects_dir, accept=True)
labels = mne.read_labels_from_annot('fsaverage', 'HCPMMP1', 'lh', subjects_dir=subjects_dir)
labels_combined = mne.read_labels_from_annot('fsaverage', 'HCPMMP1_combined', 'lh', subjects_dir=subjects_dir)

Reading labels from parcellation...
   read 181 labels from C:\Users\dalto\mne_data\MNE-sample-data\subjects\fsaverage\label\lh.HCPMMP1.annot
Reading labels from parcellation...
   read 23 labels from C:\Users\dalto\mne_data\MNE-sample-data\subjects\fsaverage\label\lh.HCPMMP1_combined.annot


In [34]:
with h.File(raw,'r+') as f:
    orig_det = f['nirs/probe/detectorPos3D'][:,:]
    orig_sou = f['nirs/probe/sourcePos3D'][:,:]
    f.close()

edited_det = pd.DataFrame(orig_det)
edited_det[1] += 0

edited_sou = pd.DataFrame(orig_sou)
edited_sou[1] += 0

with h.File(raw,'r+') as f:
    f['nirs/probe/detectorPos3D'][:,:] = edited_det
    f['nirs/probe/sourcePos3D'][:,:] = edited_sou
    # edited_det[1] += 10
    # edited_det[2] += 20

    # edited_sou[1] += 10
    # edited_sou[2] += 20
    f['nirs/probe/detectorPos3D'][:,:] = edited_det
    f['nirs/probe/sourcePos3D'][:,:] = edited_sou

    # print(f"\n{f['nirs/probe/detectorPos3D'][:,:] == orig_det}")
    f.close()

print(edited_det)

raw_intensity = mne.io.read_raw_snirf(raw, verbose=False)

           0         1         2
0  -0.066571  0.081416  0.047017
1  -0.064767  0.058143  0.090843
2  -0.085904  0.018972  0.065098
3  -0.088178  0.000578  0.046986
4  -0.085987 -0.015713  0.029306
5  -0.071206 -0.012874  0.107879
6  -0.072622 -0.045692  0.069671
7   0.062798  0.081766  0.047531
8   0.060651  0.058824  0.091177
9   0.081887  0.020428  0.065713
10  0.084072  0.001807  0.047866
11  0.082579 -0.014900  0.029868
12  0.067128 -0.012199  0.108573
13  0.069030 -0.045194  0.070324
14 -0.047447  0.095404  0.073381
15 -0.076963  0.058318  0.056967
16 -0.086154 -0.006278  0.068867
17 -0.065077 -0.040018  0.100009
18  0.054578  0.083417  0.073752
19  0.077336  0.044960  0.058661
20  0.078973 -0.021052  0.070573
21  0.050798 -0.052066  0.100971


In [40]:
brain = mne.viz.Brain(
    'fsaverage', subjects_dir=subjects_dir, background='w', cortex='low_contrast', hemi='both')
#defines brain as the command that activates the freeview mode

brain.add_sensors(
    raw_intensity.info, trans='fsaverage',
    fnirs=['pairs','channels','sources', 'detectors'])
#if you want to input the channels in the figure swap this line for "fnirs=['channels', 'pairs', 'sources', 'detectors'])"

brain.show_view(azimuth=200, elevation=60, distance=400, view='rostral')
#runs the freeview mode

Channel types::	fnirs_cw_amplitude: 92


In [36]:
data_path = sample.data_path()
subjects_dir = data_path / "subjects"
sample_dir = data_path / "MEG" / "sample"

In [37]:
brain_kwargs = dict(alpha=0.1, background="white", cortex="low_contrast")
brain = mne.viz.Brain("sample", subjects_dir=subjects_dir, **brain_kwargs)

stc = mne.read_source_estimate(sample_dir / "sample_audvis-meg")
stc.crop(0.09, 0.1)

kwargs = dict(
    fmin=stc.data.min(),
    fmax=stc.data.max(),
    alpha=0.25,
    smoothing_steps="nearest",
    time=stc.times,
)
brain.add_data(stc.lh_data, hemi="lh", vertices=stc.lh_vertno, **kwargs)
brain.add_data(stc.rh_data, hemi="rh", vertices=stc.rh_vertno, **kwargs)


In [38]:
brain = mne.viz.Brain("sample", subjects_dir=subjects_dir, **brain_kwargs)
brain.show_view(azimuth=190, elevation=70, distance=350, focalpoint=(0, 0, 20))

In [39]:
brain = mne.viz.Brain("sample", subjects_dir=subjects_dir, **brain_kwargs)
brain.add_label("BA44", hemi="lh", color="green", borders=True)
brain.show_view(azimuth=190, elevation=70, distance=350, focalpoint=(0, 0, 20))